In [16]:
import pandas as pd
import numpy as np
import time
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns


caminho_do_projeto = r'C:\Users\kmenezes\OneDrive - unb.br\tcc_organizacao'

if caminho_do_projeto not in sys.path:
    sys.path.insert(0, caminho_do_projeto)

from projeto_aplicado.modelos.otimizador_pli import OtimizadorPLI
from projeto_aplicado.modelos.otimizador_aco import OtimizadorACO
from projeto_aplicado.modelos.otimizador_ag import OtimizadorAG
from projeto_aplicado.modelos.otimizador_base import Otimizador
from projeto_aplicado.modelos.analisador import AnalisadorDeSolucao

# Configurações de visualização
sns.set_theme(style="whitegrid")
pd.set_option('display.max_rows', 100)

In [17]:
CONFIG_BASE = {
    "ARQUIVOS_DADOS": {
        "disciplinas": "disciplinas.csv",
        "professores": "docentes.csv",
        "preferencias": "preferencias.csv",
        "conflitos": "matriz_conflitos.csv" 
    },
    "ALOCACOES_FIXAS": [
    ]
}

# Parâmetros específicos para cada algoritmo
CONFIG_PLI = {
    **CONFIG_BASE,
    "PENALIDADE_W": 4.0
}

CONFIG_ACO = {
    **CONFIG_BASE,
    "ACO_PARAMS": {
        "n_formigas": 20,
        "n_geracoes": 30,
        "alfa": 3.0,
        "beta": 4.0,
        "taxa_evaporacao": 0.3,
    }
}

CONFIG_AG = {
    **CONFIG_BASE,
    "AG_PARAMS": {
        "n_populacao": 100,
        "n_geracoes": 50,
        "taxa_crossover": 0.85,
        "taxa_mutacao": 0.15,
        "tamanho_torneio": 8,
        "fator_penalidade": 10000,
        "tamanho_elite": 10
    }
}
analise = AnalisadorDeSolucao(config=CONFIG_BASE)
otimizador = Otimizador(config=CONFIG_BASE)
d1 = otimizador.dados_brutos
prof = d1['professores']
disc = d1['disciplinas']

df = pd.DataFrame()

Dados de contexto carregados pelo analisador.
Analisador de Solução autônomo pronto.
Buscando dados no diretório: C:\Users\kmenezes\OneDrive - unb.br\tcc_organizacao\dados
Dados brutos carregados.
0 alocações fixas validadas e separadas.
28 de 28 professores responderam ao formulário.
Dados preparados. O problema foi reduzido para a otimização.
Otimizador base inicializado. 0 alocações foram fixadas.


In [18]:
s252 = pd.read_csv('../dados/solucao_25_1.csv')
rdep = s252.merge(disc[['id_disciplina', 'disciplina']], on='id_disciplina', how='left').merge(prof[['id_docente', 'docente']], on='id_docente',how='left')
rdep2 = rdep[['id_disciplina', 'docente', 'preferencia']]
rdep2.columns = ['id_disciplina',  'dep_docente', 'dep_preferencia']
result = analise.avaliar(rdep)
df1 = pd.DataFrame({'metodo':['Departamento'],'escore_total': result['escore_total'], **result['distribuicao_preferencias']})
df = pd.concat([df, df1], ignore_index=True)

In [19]:
otimizador_pli = OtimizadorPLI(config=CONFIG_PLI)
inicio = time.perf_counter()
rpli = otimizador_pli.resolver()['alocacao_final']
duracao_pli = time.perf_counter() - inicio
rpli = rpli.merge(prof[['id_docente', 'docente']], on ='id_docente',how='left')
rpli2 = rpli[['id_disciplina', 'docente', 'preferencia']]
rpli2.columns = ['id_disciplina',  'pli_docente', 'pli_preferencia']
print(analise.avaliar(rpli))

result = analise.avaliar(rpli)
df1 = pd.DataFrame({'metodo':['PLI'],
                    'escore_total': result['escore_total'],
                    'tempo_execucao': duracao_pli,
                    **result['distribuicao_preferencias']})
df = pd.concat([df, df1], ignore_index=True)

Buscando dados no diretório: C:\Users\kmenezes\OneDrive - unb.br\tcc_organizacao\dados
Dados brutos carregados.
0 alocações fixas validadas e separadas.
28 de 28 professores responderam ao formulário.
Dados preparados. O problema foi reduzido para a otimização.
Otimizador base inicializado. 0 alocações foram fixadas.
OtimizadorPLI pronto.

--- Iniciando Processo de Otimização ---
Modelo PLI construído com sucesso.

Iniciando a resolução do modelo PLI...
Status da Solução: Optimal
Solução extraída e formatada.
--- Processo de Otimização Concluído ---
{'escore_total': np.int64(136), 'distribuicao_preferencias': {'pref_3': np.int64(43), 'pref_2': np.int64(3), 'pref_1': np.int64(1), 'pref_0': 0}}


In [20]:
otimizador_aco = OtimizadorACO(config=CONFIG_ACO)
inicio = time.perf_counter()
raco = otimizador_aco.resolver()['alocacao_final']
duracao_aco = time.perf_counter() - inicio

raco = raco.merge(prof[['id_docente', 'docente']], on ='id_docente',how='left')
raco2 = raco[['id_disciplina', 'docente', 'preferencia']]
raco2.columns = ['id_disciplina',  'aco_docente', 'aco_preferencia']
result = analise.avaliar(raco)
df1 = pd.DataFrame({'metodo':['ACO'],
                    'escore_total': result['escore_total'],
                    'tempo_execucao': duracao_aco,
                    **result['distribuicao_preferencias']})
df = pd.concat([df, df1], ignore_index=True)

Buscando dados no diretório: C:\Users\kmenezes\OneDrive - unb.br\tcc_organizacao\dados
Dados brutos carregados.
0 alocações fixas validadas e separadas.
28 de 28 professores responderam ao formulário.
Dados preparados. O problema foi reduzido para a otimização.
Otimizador base inicializado. 0 alocações foram fixadas.
OtimizadorACO pronto.

--- Iniciando Processo de Otimização ---
Matrizes de feromônio e heurística inicializadas.
Geração 10/30 | Melhor Global: 135
Geração 20/30 | Melhor Global: 135
Geração 30/30 | Melhor Global: 135
Otimização ACO concluída.
--- Processo de Otimização Concluído ---


In [21]:
otimizador_ag = OtimizadorAG(config=CONFIG_AG)
inicio = time.perf_counter()
rag = otimizador_ag.resolver()['alocacao_final']
duracao_ag = time.perf_counter() - inicio
rag = rag.merge(prof[['id_docente', 'docente']], on ='id_docente',how='left')
rag2 = rag[['id_disciplina', 'docente', 'preferencia']]
rag2.columns = ['id_disciplina',  'ag_docente', 'ag_preferencia']
result = analise.avaliar(rag)
df1 = pd.DataFrame({'metodo':['AG'],
                    'escore_total': result['escore_total'],
                    'tempo_execucao': duracao_ag,
                    **result['distribuicao_preferencias']})
df = pd.concat([df, df1], ignore_index=True)

Buscando dados no diretório: C:\Users\kmenezes\OneDrive - unb.br\tcc_organizacao\dados
Dados brutos carregados.
0 alocações fixas validadas e separadas.
28 de 28 professores responderam ao formulário.
Dados preparados. O problema foi reduzido para a otimização.
Otimizador base inicializado. 0 alocações foram fixadas.
OtimizadorAG pronto.

--- Iniciando Processo de Otimização ---
População inicial com 100 indivíduos criada, respeitando os genes fixos.
Geração 20/50 | Melhor Global: 135.00
Geração 40/50 | Melhor Global: 136.00
Otimização AG concluída.
--- Processo de Otimização Concluído ---


In [22]:
disc = disc[['id_disciplina', 'disciplina']].merge(rdep2, on = 'id_disciplina', how='left')\
                                    .merge(rpli2, on = 'id_disciplina', how='left')\
                                    .merge(raco2, on = 'id_disciplina', how='left')\
                                    .merge(rag2, on = 'id_disciplina', how='left')

disc.to_excel('comparacao_resultados.xlsx',index=False)

In [23]:
df

,metodo,escore_total,pref_3,pref_2,pref_1,pref_0,tempo_execucao
0,Departamento,124,40,1,2,4,NaN
1,PLI,136,43,3,1,0,3.874404
2,ACO,135,42,4,1,0,17.602127
3,AG,136,43,3,1,0,5.654272


In [24]:
rpli[rpli['preferencia'] != 3]

,id_disciplina,id_docente,preferencia,ch_disciplina,docente
2,EST0088_01,docente_02,2,1,ANA MARIA NOGALES VASCONCELOS
6,EST0091_01,docente_06,2,1,DEMERSON ANDRE POLLI
10,EST0124_01,docente_08,2,1,EDUARDO MONTEIRO DE CASTRO GOMES
40,EST0046_01,docente_25,1,1,PETER ZORNIG


In [25]:
disc[disc['pli_preferencia'] != 3]

,id_disciplina,disciplina,dep_docente,dep_preferencia,pli_docente,pli_preferencia,aco_docente,aco_preferencia,ag_docente,ag_preferencia
34,EST0046_01,DEMOGRAFIA,ANA MARIA NOGALES VASCONCELOS,0.0,PETER ZORNIG,1,PETER ZORNIG,1,PETER ZORNIG,1
43,EST0088_01,PRATICAS DE EXTENSAO EM ESTATISTICA 2,LUCAS MOREIRA,0.0,ANA MARIA NOGALES VASCONCELOS,2,ANA MARIA NOGALES VASCONCELOS,2,ANA MARIA NOGALES VASCONCELOS,2
44,EST0091_01,COMPUTACAO EM ESTATISTICA 2: R,DONALD MATTHEW PIANTO,1.0,DEMERSON ANDRE POLLI,2,DEMERSON ANDRE POLLI,2,DEMERSON ANDRE POLLI,2
46,EST0124_01,LABORATORIO DE ESTATISTICA 1,JOANLISE MARCO DE LEON ANDRADE,1.0,EDUARDO MONTEIRO DE CASTRO GOMES,2,EDUARDO MONTEIRO DE CASTRO GOMES,2,ANA MARIA NOGALES VASCONCELOS,2


In [32]:
disc

,id_disciplina,disciplina,dep_docente,dep_preferencia,pli_docente,pli_preferencia,aco_docente,aco_preferencia,ag_docente,ag_preferencia
0,EST0001_01,COMPUTACAO EM ESTATISTICA 1,JAMES MATOS SAMPAIO,3.0,DONALD MATTHEW PIANTO,3,FELIPE SOUSA QUINTINO,2,JAMES MATOS SAMPAIO,3
1,EST0001_02,COMPUTACAO EM ESTATISTICA 1,JAMES MATOS SAMPAIO,3.0,JAMES MATOS SAMPAIO,3,JAMES MATOS SAMPAIO,3,JAMES MATOS SAMPAIO,3
2,EST0004_01,ESTATISTICA COMPUTACIONAL,HELTON SAULO BEZERRA DOS SANTOS,3.0,HELTON SAULO BEZERRA DOS SANTOS,3,HELTON SAULO BEZERRA DOS SANTOS,3,HELTON SAULO BEZERRA DOS SANTOS,3
3,EST0005_01,INFERENCIA BAYESIANA,GUSTAVO LEONEL GILARDONI AVALLE,1.0,DONALD MATTHEW PIANTO,3,DONALD MATTHEW PIANTO,3,DONALD MATTHEW PIANTO,3
4,EST0011_01,MODELOS LINEARES GENERALIZADOS,TEREZINHA KESSIA DE ASSIS RIBEIRO,3.0,TEREZINHA KESSIA DE ASSIS RIBEIRO,3,FREDERICO MACHADO ALMEIDA,3,TEREZINHA KESSIA DE ASSIS RIBEIRO,3
5,EST0017_01,METODOS ESTATISTICOS 2,LUIS GUSTAVO DO AMARAL VINHA,3.0,EDUARDO YOSHIO NAKANO,3,EDUARDO YOSHIO NAKANO,3,EDUARDO YOSHIO NAKANO,3
6,EST0019_01,ESTATISTICA APLICADA,DEMERSON ANDRE POLLI,0.0,JOANLISE MARCO DE LEON ANDRADE,3,NICOLLAS STEFAN SOARES DA COSTA,3,JOANLISE MARCO DE LEON ANDRADE,3
7,EST0019_02,ESTATISTICA APLICADA,JOSE ANGELO BELLONI,1.0,ANA MARIA NOGALES VASCONCELOS,3,JOANLISE MARCO DE LEON ANDRADE,3,LUIS GUSTAVO DO AMARAL VINHA,3
8,EST0019_03,ESTATISTICA APLICADA,ANA MARIA NOGALES VASCONCELOS,0.0,JOSE ANGELO BELLONI,3,GUSTAVO LEONEL GILARDONI AVALLE,3,GUSTAVO LEONEL GILARDONI AVALLE,3
9,EST0019_04,ESTATISTICA APLICADA,JOSE ANGELO BELLONI,1.0,GUSTAVO LEONEL GILARDONI AVALLE,3,LUIS GUSTAVO DO AMARAL VINHA,3,DEMERSON ANDRE POLLI,3


In [35]:
rpli['id_docente'].unique()

array(['docente_01', 'docente_02', 'docente_04', 'docente_06',
       'docente_07', 'docente_08', 'docente_09', 'docente_10',
       'docente_11', 'docente_12', 'docente_13', 'docente_14',
       'docente_15', 'docente_16', 'docente_17', 'docente_18',
       'docente_19', 'docente_20', 'docente_21', 'docente_22',
       'docente_23', 'docente_24', 'docente_25', 'docente_26',
       'docente_27', 'docente_28'], dtype=object)

In [45]:
rag['id_docente'].nunique()

26

In [47]:
disc['id_disciplina'].nunique()

47